## Desarrollo con el método DAISY
El descriptor de imagen local DAISY se basa en histogramas de orientación de gradiente similares al descriptor SIFT. Está formulado de una manera que permite una extracción densa rápida que es útil para representar carácterísticas de imágenes

### Carga de bibliotecas


In [ ]:
import cv2
import numpy as np
import os
from skimage.feature import daisy

### Lectura del dataset (dentro del entorno colab) y Aplicación de DAISY

In [ ]:
train_path = 'Animales/Training'
training_names = os.listdir(train_path)


image_paths = []
#Etiquetas
image_classes = []

class_id = 0


def imglist(path):
    return [os.path.join(path,f) for f in os.listdir(path)]


for training_name in training_names:
    dir = os.path.join(train_path, training_name)
    class_path = imglist(dir)
    image_paths += class_path
    image_classes += [class_id]*len(class_path)
    class_id += 1

caractDAISY = []

for image_path in image_paths:
    im = cv2.imread(image_path)
    gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    dDaysi, des = daisy(gray, step=180, radius=50, rings=2, histograms=6,
                         orientations=8, visualize=True)
    
    rsdDaysi = np.reshape(dDaysi,(dDaysi.shape[0]*dDaysi.shape[1],dDaysi.shape[2]))
    
    caractDAISY.append((image_path,rsdDaysi))


descriptors = caractDAISY[0][1]


In [ ]:
for image_path, descriptor in caractDAISY[1:]:
    descriptors = np.vstack((descriptors, descriptor))

descriptors = descriptors.astype(float)

### Bag of words

In [ ]:
#Kmeans

from scipy.cluster.vq import kmeans ,vq

k = 60

voc,variance = kmeans(descriptors,k,1)

im_features = np.zeros((len(image_paths),k), "float32")

for i in range(len(image_paths)):
    words, distance = vq(caractDAISY[i][1],voc)
    for w in words:
        im_features[i][w] += 1

### Agregar columna de clases y creación de Dataframe

In [ ]:
y = np.array(image_classes)

x = np.array(im_features)

import pandas as pd

Data = pd.DataFrame(x) 

#60 Etiqueta

Data['Clase'] = y
Data = Data.rename(columns = {'60':'Clase'})


Data.to_csv('Daisy.csv', header = True, index= True)



In [ ]:
Data

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,Clase
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,4.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20.0,0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3
188,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3
190,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3


## Conclusiones

1. Después de comparar SIFT,SURF y DAISY podemos decir que SIFT es el algoritmo más rápido.

2. Se utilizaron 50 imágenes divididas de animales en 4 carpetas y la demora en ejecución es porque debíamos hacer un reescalado a la imagen, pero esto podría resultar en perdida de información de las imágenes.

3. Al probar Daisy se observó que el vector de características tenía valores muy pequeños y la mayoría eran 0.

4. Hubo dificuldad a la hora de hallar el método Daisy, debido a que no se encontraba dentro de la biblioteca opencv (no se encontraba declarada). Por ello se buscó la alternativa de skimage